In [1]:
import ollama
import gradio as gr
import base64
from PIL import Image
import io

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def encode_image(image_path):
    """Encode image to base64 string"""
    with open(image_path, "rb") as image_file:     # rb: 바이너리 읽기(이미지는 0,1 형태의 바이너리)
        return base64.b64encode(image_file.read()).decode('utf-8')

In [3]:
def analyze_image_with_gemma(image):
    """Ollama Gemma 3 4B model을 사용하여 이미지 분석"""
    try:
        # Convert Gradio image to PIL Image
        pil_image = Image.fromarray(image)   # PIL(이미지처리, 조작) 이미지 객체로 변환

        # Save image to a temporary buffer
        buffer = io.BytesIO()  # 메모리 상에 임시 버퍼를 만들어서 이미지 데이터 저장
        pil_image.save(buffer, format="PNG")

        # Encode image to base64
        encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')  # 임시 버퍼에 저장된 이미지 데이터를 바이트 문자열로 가져옴

        # Perform image analysis using Ollama Gemma3
        response = ollama.chat(
            model='gemma3:latest',
            messages=[
                {
                    'role': 'user',
                    'content': '이미지의 내용을 자세히 설명해줘. 무엇이 보이는지, 색상, 구성, 감정 등을 포함해서 분석해줘.', #prompt
                    'images': [encoded_image]
                }
            ]
        )

        return response['message']['content']

    except Exception as e:
        return f"이미지 분석 중 오류 발생: {str(e)}"

In [4]:
iface = gr.Interface(
        fn=analyze_image_with_gemma,
        inputs=gr.Image(type="numpy", label="이미지 업로드"),
        outputs=gr.Textbox(label="이미지 분석 결과"),
        title="Ollama Gemma3 이미지 분석기",
        description="이미지를 업로드하면 Gemma 3 4B 모델이 분석해드립니다."
    )

In [5]:
iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [6]:
iface.close()

Closing server running on port: 7860
